In [5]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd


from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime

## 1. 상품 코드 추출

In [38]:
def KewordProductCodeCrawling(keyword_name):
    print('##### 키워드 = "{}"의 상품 코드 추출을 시작합니다. ########'.format(keyword_name))
    driver = webdriver.Chrome('/Users/dongyokim/main/jupyter/chromedriver/chromedriver')
    
    driver.set_window_size(1000, 1000)

    driver.get('https://www.glowpick.com/searches/keywords')

    time.sleep(1)
    search = driver.find_element(By.CSS_SELECTOR, '#default-layout > div > div.glowpick-layer > div > div > div > input')

    # 카테고리 이름 입력 
    search.send_keys(keyword_name)

    time.sleep(0.5)

    # 엔터
    search.send_keys(Keys.ENTER)
    
    # 원하는 위치 스크롤 내리기  
    # finish_line = 스킨 40000 기준: 260 개
    # 로션 40000 기준: 360개
    # 크림 40000 기준: 340
    finish_line = 40000
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        # 우선 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(0.5)       # 작업 중간에 1이상으로 간격을 줘야 데이터 취득가능(스크롤을 내릴 때의 데이터 로딩 시간 때문)
        # 현재 위치 담기
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        # 과거의 길이와 현재 위치 비교하기
        if new_page_height > finish_line:
            break
        else: 
            last_page_height = new_page_height


    # 스크롤을 내린 후에 모든 상품들의 코드를 담은 리스트 find 
    time.sleep(0.5)
    products_code = driver.find_elements(By.CSS_SELECTOR, 'div.products__product')
    time.sleep(0.5)
    print('#### 총 {}개의 제품의 상품 코드가 추출되었습니다. ###### '.format(len(products_code)))
    total_code = []

    # 반복문 돌려서 각각 제품의 id만 추추(product-#####) -을 기준으로 split으로 쪼갠다음에 번호만 뽑아내자
    for code in products_code:
        product_id = code.get_attribute('id').split('-')[1]
        total_code.append(product_id)
    
    print('####### 키워드 = "{}"의 상품 코드 크롤링을 종료합니다. ########'.format(keyword_name))
    print('------------------------------------------------')
    print()
    driver.quit()
    
    return total_code

In [39]:
# 각 키워드 설정 
category_name = ['스킨', '로션', '크림']

total_product_code_data = []
for name in category_name:

    product_code_data = KewordProductCodeCrawling(name)
    total_product_code_data.append(product_code_data)


##### 키워드 = "스킨"의 상품 코드 추출을 시작합니다. ########
#### 총 260개의 제품의 상품 코드가 추출되었습니다. ###### 
####### 키워드 = 스킨의 상품 코드 크롤링을 종료합니다. ########
------------------------------------------------

##### 키워드 = "로션"의 상품 코드 추출을 시작합니다. ########
#### 총 360개의 제품의 상품 코드가 추출되었습니다. ###### 
####### 키워드 = 로션의 상품 코드 크롤링을 종료합니다. ########
------------------------------------------------

##### 키워드 = "크림"의 상품 코드 추출을 시작합니다. ########
#### 총 340개의 제품의 상품 코드가 추출되었습니다. ###### 
####### 키워드 = 크림의 상품 코드 크롤링을 종료합니다. ########
------------------------------------------------



In [44]:
# 딕셔너리 형태로 저장
product_code_dic = {'스킨': total_product_code_data[0], 
    '로션': total_product_code_data[1], 
    '크림': total_product_code_data[2]}

## 2. 상품 정보 추출

In [137]:
def GlowPickCrawling(code_dic): 
    
    
    driver = webdriver.Chrome('/Users/dongyokim/main/jupyter/chromedriver/chromedriver')
    driver.set_window_size(1000, 1000)
    
    key_name = list(code_dic.keys())[0]
    
    total_product_list = []
    print('##################')
    print('글로우픽 화장품 정보 크롤링을 시작합니다.')
    print('###### 카테고리 = {}의 상품 정보를 추출합니다. ######'.format(key_name))
    
    for i in code_dic[key_name][:5]:
        driver.get('https://www.glowpick.com/products/{}'.format(i))
        time.sleep(0.5)

        # 화장품 기초 정보
        try:
            product_brand_name = driver.find_element(By.CSS_SELECTOR, 'p.product__summary__brand').text
        except:
            continue

        # 화장품 이름(없을 시 바로 다음 페이지로 이동)
        try:
            product_name = driver.find_element(By.CSS_SELECTOR, 'p.product__summary__name').text
        except:
            continue
            

        try:
            product_volume = driver.find_element(By.CSS_SELECTOR, 'p.product__summary__offer.offer > span.offer__volume-price.font-spoqa').text.split('/')[0]
        except:
            product_volume = ''
        try:
            product_price = driver.find_element(By.CSS_SELECTOR, 'p.product__summary__offer.offer > span.offer__volume-price.font-spoqa').text.split('/')[1]
        except:
            product_price = ''
        try:
            product_rating = driver.find_element(By.CSS_SELECTOR, 'p.product__summary__offer.offer > span.offer__rating-review > span.stars.offer__stars > span.stars__rating').text
        except:
            product_rating = ''
        try:
            product_review_count = driver.find_element(By.CSS_SELECTOR, 'p.product__summary__offer.offer > span.offer__rating-review > span.offer__reviews').text.split(' ')[1]
        except:
            product_review_count = ''

        # 화장품 상세정보 
        try:
            # 화장품 종류
            product_category = driver.find_element(By.CSS_SELECTOR, 'div.product__info.info > article.info__article.rank-pd > ul > li:nth-child(1) > span.rank-item__name').text
        except:
            product_category = ''

        try:
            # 카테고리 내 순위 
            product_category_rank = driver.find_element(By.CSS_SELECTOR, 'div.product__info.info > article.info__article.rank-pd > ul > li:nth-child(1) > span.rank-item__rank').text
        except:
            product_category_rank = ''

        try:
            # 브랜드 내 순위
            product_brand_rank = driver.find_element(By.CSS_SELECTOR, 'div.product__info.info > article.info__article.rank-pd > ul > li:nth-child(2) > span.rank-item__rank').text
        except:
            product_brand_rank = ''


        try:
            # 제품 설명
            product_description = driver.find_element(By.CSS_SELECTOR, 'article.info__article.description > div > pre').text
        except:
            product_description = ''

        try:
            # 제품 만족도 
            product_ratings_value = driver.find_element(By.CSS_SELECTOR, 'div.reviews__summary__ratings.ratings > p.ratings__value > span.ratings__value__ratio.font-spoqa').text
        except:
            product_ratings_value = ''
        
        # 성분구성표 클릭 
        try:
            ingredient_click = driver.find_element(By.CSS_SELECTOR, 'div.product__info.info > article.info__article.ingredient > h3 > button')
            time.sleep(0.5)
            ingredient_click.send_keys(Keys.ENTER)
        

            # 각 위험도별 성분 개수 
            unknown_ingredient_count = driver.find_element(By.CSS_SELECTOR, 'div.details__contents.awards > div > article:nth-child(1) > div > div:nth-child(1) > div > span').text
            low_ingredient_count = driver.find_element(By.CSS_SELECTOR, 'div.details__contents.awards > div > article:nth-child(1) > div > div:nth-child(2) > div > span').text
            middle_ingredient_count = driver.find_element(By.CSS_SELECTOR, 'div.details__contents.awards > div > article:nth-child(1) > div > div:nth-child(3) > div > span').text
            high_ingredient_count = driver.find_element(By.CSS_SELECTOR, 'div.details__contents.awards > div > article:nth-child(1) > div > div:nth-child(4) > div > span').text

            # 세부 성분 이름 
            ingredient_list = driver.find_elements(By.CSS_SELECTOR, 'div.details__contents.awards > div > ul > li')
            ingredient_name_list = []
            for ingredient in ingredient_list:
                kor_name = ingredient.find_element(By.CSS_SELECTOR, 'div > div.item__wrapper__text > p.item__wrapper__text__kor').text    
                ingredient_name_list.append(kor_name)
                
        except:
            unknown_ingredient_count = ''
            low_ingredient_count = ''
            middle_ingredient_count = ''
            high_ingredient_count = ''
            ingredient_name_list = ''
        
        
            
        total_product_list.append([product_name, product_brand_name, product_brand_rank,product_volume, product_price,  product_rating, product_ratings_value, product_review_count, product_category, product_category_rank,  
                                   unknown_ingredient_count, low_ingredient_count, middle_ingredient_count,high_ingredient_count, 
                                   ingredient_name_list,product_description])
        
        time.sleep(0.5)
    driver.quit() 
    print('###########################')
    print('{}개의 키워드 = "{}"의 상품 정보 수집 완료'.format(len(code_dic[key_name]), key_name))
    print('chrome driver를 종료합니다.')
    print('###########################')
    print('----------------------------')
    driver.quit()
    
    
    df = pd.DataFrame(data=total_product_list, columns=['화장품명', '브랜드명', '브랜드 내 순위', '용량', '가격', '평점', '만족도', '리뷰 수', '카테고리', '카테고리 내 순위', 
                                                        '위험도 미정 성분 개수', 
                                                        '낮은 위험도 성분 개수', '중간 위험도 성분 개수', '높은 위험도 성분 개수','구성 성분 리스트','제품 설명'])
    

    return df

In [138]:
data = GlowPickCrawling(product_code_dic)

##################
글로우픽 화장품 정보 크롤링을 시작합니다.
###### 카테고리 = 스킨의 상품 정보를 추출합니다. ######
###########################
260개의 키워드 = "스킨"의 상품 정보 수집 완료
chrome driver를 종료합니다.
###########################
----------------------------


In [139]:
data.head()

,화장품명,브랜드명,브랜드 내 순위,용량,가격,평점,만족도,리뷰 수,카테고리,카테고리 내 순위,위험도 미정 성분 개수,낮은 위험도 성분 개수,중간 위험도 성분 개수,높은 위험도 성분 개수,구성 성분 리스트,제품 설명
0,DAC2 알래스카 엘엠더블유 히알루로닉 토너,닥터릴리프,2위,205ml,"39,000원",4.64,96%,232,워터토너,1위,1개,17개,0개,0개,"[빙하수, 글리세레스-26, 버지니아풍년화추출물, 부틸렌글라이콜, 판테놀, 병풀추출...","DAC2포뮬러에 저분자 히알루론산, 알래수카 빙하수, 위치하젤 추출물로 채워진 더마..."
1,파워 10 포뮬라 엘아이 젤리 패드 감초줄렌,잇츠스킨,1위,70ea,"22,000원",4.63,96%,534,토너패드,2위,3개,27개,1개,0개,"[정제수, 부틸렌글라이콜, 글리세린, 스페인감초뿌리추출물, 알란토인, 인도멀구슬나무...",촉촉한 수분감이 빈틈없이 밀착되는 팩 겸용 진정 패드\n\n- 건조한 피부를 편안하...
2,락토크림 버블워시 [머스크 라벤더],스킨유,1위,600ml,"18,900원",4.60,95%,191,바디워시,1위,0개,25개,8개,3개,"[정제수, 글리세린, 다이프로필렌글라이콜, 솔비톨, 라우릴베타인, 다마스크장미꽃수,...",초고밀도 크림거품의 프로바이오틱스 샤워케어\n\n- 한번 펌핑으로 전신샤워\n- 까...
3,마데카소사이드 플루이드 2X,어퓨,2위,300ml,"16,000원",4.62,94%,172,에센스토너,1위,4개,29개,5개,0개,"[정제수, 병풀잎수, 다이프로필렌글라이콜, 글리세린, 1,2-헥산다이올, 피피지-1...",* 본 제품은 마데카소사이드 플루이드에서 리뉴얼된 제품입니다.\n\n2배 더 강화된...
4,파워 10 포뮬라 엘아이 이펙터 감초줄렌,잇츠스킨,2위,30ml,"15,400원",4.54,91%,"1,376",진정에센스,1위,1개,23개,1개,0개,"[스페인감초뿌리추출물, 정제수, 프로판다이올, 글리세린, 베타인, 마트리카리아꽃추출...",* 본 제품은 파워 10 포뮬라 엘아이 이펙터에서 리뉴얼된 제품입니다.\n\n민감한...


In [140]:
# 현재 날짜 기준으로 저장
today_date = datetime.today().strftime("%Y_%m_%d") 

# 데이터를 원하는 형식으로 저장(csv일 경우 to_csv)
data.to_excel('GlowPic_dataset_{}.xlsx'.format(today_date))